# Imports & setup

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
from pathlib import Path
from dotenv import load_dotenv
import os 
import pandas as pd
from datetime import datetime

# user made files 
from utilities.workshift_data import SHEET_TO_WORKSHIFT_COLS, GOOGLE_SHEET_COL_TYPES
from classes.shiftClass import WorkShift

SCRIPTS_DIR = Path.cwd()

In [40]:
try: 
    env_path = SCRIPTS_DIR / ".env"
    load_dotenv(dotenv_path=env_path)  # automatically looks for .env in the scripts directory

    creds_path = SCRIPTS_DIR / os.getenv("GOOGLE_SHEETS_CREDS_FILE")
    
    # Define the scopes
    scopes = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ]

    # Authorize
    creds = Credentials.from_service_account_file(creds_path, scopes=scopes)
    client = gspread.authorize(creds)

    # -----------------------------
    # Open sheet by name or URL
    # -----------------------------
    sheet = client.open("Copy of Staples Finances 2025").sheet1
    data = sheet.get_all_values()

    print("Successfully connected to sheet.")
    print(f"Number of rows: {len(data)}")



except Exception as e: 
    print(f"{e}")

Successfully connected to sheet.
Number of rows: 52


# Dataframe

In [34]:
# Create DataFrame using first row as header
df = pd.DataFrame(data[1:], columns=data[0])
print(df)


                                                              YEAR SELECTOR  \
0                                      Variables                              
1                                        Payrate                     $17.80   
2                                Copy Center Pay                     $18.50   
3                                        Overpay                     $25.35   
4                                           inti                      $0.00   
5                                                                             
6                                          Notes                              
7               Pay period is Sunday to Saturday                              
8                   Overpay starts after 8 hours                              
9   Hlday: shift*pay/2 + shift + 4 * overTimePay                              
10                                                                            
11                                                  

# Map the columns of the Google Sheet

In [ ]:
colMap = {}

# Find the positions of the important columns 
for idx, col in enumerate(df.columns):
    print(f"The col \"{col}\" has the index: {idx}")

    if col in GOOGLE_SHEET_COL_TYPES:
        colMap.update({col : idx + 1})

print(f"\nThe important columns are as follows:")
print(colMap)

The col "" has the index: 0
The col "YEAR SELECTOR" has the index: 1
The col "WKLY HRS" has the index: 2
The col "" has the index: 3
The col "DATE" has the index: 4
The col "IN" has the index: 5
The col "LUNCH IN" has the index: 6
The col "LUNCH OUT" has the index: 7
The col "OUT" has the index: 8
The col "" has the index: 9
The col "time" has the index: 10
The col "hours" has the index: 11
The col "EARN" has the index: 12
The col "skip lunch" has the index: 13
The col "PAYCHCK" has the index: 14
The col "Assumed tax" has the index: 15
The col "DATE RECIEVED" has the index: 16
The col "" has the index: 17
The col "bfr tax est total:" has the index: 18
The col "$4,046.67" has the index: 19
The col "paychck tot:" has the index: 20
The col "$3,206.94" has the index: 21
The col "Difference:" has the index: 22
The col "$839.73" has the index: 23
The col "21%" has the index: 24

The important columns are as follows:
{'DATE': 5, 'IN': 6, 'LUNCH IN': 7, 'LUNCH OUT': 8, 'OUT': 9, 'bfr tax est t

# Create a new shift

In [36]:
newShift = WorkShift(clock_in=datetime(2025, 7, 25, 16), clock_out=datetime(2025, 7, 25, 20, 30), lunch_in=datetime(2025, 7, 25, 17), lunch_out=datetime(2025, 7, 25, 17, 30), notes='this is a python shift!')
newShift.view


IN: 	Fri Jul 25	4:00 PM
OUT: 	Fri Jul 25	8:30 PM
Lunch start: 	Fri Jul 25	5:00 PM
Lunch end: 	Fri Jul 25	5:30 PM
Shift length: 	4.00
Payrate: 	staples copy center
Payrate: 	$18.50 per hour
Pre-tax: 	$74.00



# Find the next row to update 

In [37]:
# Find the next row in the google sheet to update

# first find the next empty row 
nextEmptyGoogleSheetRow = [] # this will always be saved as the row indexing system used by Google Sheets
                            # So --> [2, len(sheet)]

for col in colMap: 
    rowIndex = 0
    cell = df[col].iloc[rowIndex]

    while not cell: 
        rowIndex += 1
        cell = df[col].iloc[rowIndex]
        if cell and (col =='IN' or col == 'OUT'): 
            nextEmptyGoogleSheetRow.append(rowIndex + 2)
            print(f"The first non-blank cell in {col} is: \t{cell}, \tat index \t{rowIndex+2}")


# if the google sheet is not ordered right, pick the lowest row number --> this overwrites the out-of-place punch
# else if the the sheet is ordered right pick the row before the first clock in clock out
if len(set(nextEmptyGoogleSheetRow)) > 1:
    nextEmptyGoogleSheetRow = min(nextEmptyGoogleSheetRow)
else: 
    nextEmptyGoogleSheetRow = min(nextEmptyGoogleSheetRow) - 1
            
print(f"So let's place a new row at row {nextEmptyGoogleSheetRow}.")

The first non-blank cell in IN is: 	10:06 AM, 	at index 	12
The first non-blank cell in OUT is: 	4:04 PM, 	at index 	12
So let's place a new row at row 11.


# Update the Google Sheet

In [39]:
# update the google sheet

numberLetterMap = {
    # Not necessary, for display only.
    5: 'E',
    6: 'F',
    7: 'G',
    8: 'H',
    9: 'I',
    19: 'S',
}

for col in colMap:
    print(f"The Google Sheet column, {col}, to update is: {numberLetterMap[colMap[col]]}")


print()


# Append a new row
for col in colMap: 
    if SHEET_TO_WORKSHIFT_COLS[col] == 'date':
        print(f"Updating: \t{numberLetterMap[colMap[col]]}{nextEmptyGoogleSheetRow}. \t{newShift.clock_in.strftime("%a %b %d")}")
        sheet.update_cell(nextEmptyGoogleSheetRow, colMap[col], newShift.clock_in.strftime("%a %b %d"))
    elif SHEET_TO_WORKSHIFT_COLS[col] == 'clock_in':
        print(f"Updating: \t{numberLetterMap[colMap[col]]}{nextEmptyGoogleSheetRow}. \t{newShift.clock_in.strftime("%-I:%M %p")}")
        sheet.update_cell(nextEmptyGoogleSheetRow, colMap[col], newShift.clock_in.strftime("%-I:%M %p"))
    elif SHEET_TO_WORKSHIFT_COLS[col] == 'lunch_in':
        print(f"Updating: \t{numberLetterMap[colMap[col]]}{nextEmptyGoogleSheetRow}. \t{newShift.lunch_in.strftime("%-I:%M %p")}")
        sheet.update_cell(nextEmptyGoogleSheetRow, colMap[col], newShift.lunch_in.strftime("%-I:%M %p"))
    elif SHEET_TO_WORKSHIFT_COLS[col] == 'lunch_out':
        print(f"Updating: \t{numberLetterMap[colMap[col]]}{nextEmptyGoogleSheetRow}. \t{newShift.lunch_out.strftime("%-I:%M %p")}")
        sheet.update_cell(nextEmptyGoogleSheetRow, colMap[col], newShift.lunch_out.strftime("%-I:%M %p"))
    elif SHEET_TO_WORKSHIFT_COLS[col] == 'clock_out':
        print(f"Updating: \t{numberLetterMap[colMap[col]]}{nextEmptyGoogleSheetRow}. \t{newShift.clock_out.strftime("%-I:%M %p")}")
        sheet.update_cell(nextEmptyGoogleSheetRow, colMap[col], newShift.clock_out.strftime("%-I:%M %p"))
    elif SHEET_TO_WORKSHIFT_COLS[col] == 'notes':
        print(f"Updating: \t{numberLetterMap[colMap[col]]}{nextEmptyGoogleSheetRow}. \t{newShift.notes}")
        sheet.update_cell(nextEmptyGoogleSheetRow, colMap[col], newShift.notes)


The Google Sheet column, DATE, to update is: E
The Google Sheet column, IN, to update is: F
The Google Sheet column, LUNCH IN, to update is: G
The Google Sheet column, LUNCH OUT, to update is: H
The Google Sheet column, OUT, to update is: I
The Google Sheet column, bfr tax est total:, to update is: S

